In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("ollama:qwen2.5:7b", temperature=0)
response = llm.invoke("Hola, como estas?")
response.text

<bound method BaseMessage.text of AIMessage(content='¡Hola! Estoy bien, gracias. ¿Y tú? ¿En qué puedo ayudarte hoy?', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-10-19T20:25:50.090238769Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6713756564, 'load_duration': 5526349542, 'prompt_eval_count': 34, 'prompt_eval_duration': 327575863, 'eval_count': 22, 'eval_duration': 848613574, 'model_name': 'qwen2.5:7b'}, id='run--e6608c38-e87d-449e-9739-6a5075d3fe58-0', usage_metadata={'input_tokens': 34, 'output_tokens': 22, 'total_tokens': 56})>

In [3]:
response = llm.invoke("Que clima hace en la ciudad de Bogota?")
response.text

<bound method BaseMessage.text of AIMessage(content='La ciudad de Bogotá, capital del país colombiano, tiene un clima tropical húmedo. Aquí te presento los detalles principales:\n\n1. Temperaturas:\n   - Promedio anual: Aproximadamente 13°C (55°F)\n   - Meses más cálidos: Junio a agosto (14-16°C)\n   - Meses más fríos: Diciembre a febrero (12-14°C)\n\n2. Humedad:\n   - Generalmente alta, con un promedio del 80% durante todo el año\n\n3. Precipitaciones:\n   - Aunque es una ciudad montañosa, Bogotá experimenta lluvias moderadas durante todo el año\n   - Las precipitaciones son más frecuentes en los meses de octubre a mayo\n\n4. Altitud:\n   - La ciudad está ubicada a unos 2.600 metros sobre el nivel del mar (8.530 pies)\n\n5. Fenómenos meteorológicos comunes:\n   - Nieve: Aunque rara, puede caer en invierno\n   - Tormentas eléctricas: Común durante la temporada lluviosa\n\n6. Estaciones:\n   - No se considera que tenga estaciones bien definidas debido a su clima tropical húmedo\n\n7. Fe

In [4]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que necesitan.

Y tus productos son:
- Computadora
- Mouse
- Teclado
- Audifonos
- Mousepad
"""
messages = [
    ("system", system_prompt),
    ("user", "Dime los productos que ofreces en la tienda")
]
response = llm.invoke(messages)
response.text

<bound method BaseMessage.text of AIMessage(content='Mis productos disponibles incluyen:\n\n1. **Computadoras**: Ofrecemos una variedad de computadoras para diferentes necesidades, desde portátiles hasta escritorios.\n2. **Mouse**: Disponemos de diversos tipos de mouse, como ópticos, inalámbricos y mecánicos, para satisfacer las preferencias individuales de los clientes.\n3. **Teclado**: Proporcionamos teclados mecánicos, ópticos y inalámbricos, adaptados a diferentes usos y preferencias.\n4. **Audífonos**: Ofrecemos una gama de audífonos para escuchar música, juegos y videoconferencias, incluyendo inalámbricos, con cable y de alta calidad.\n5. **Mousepad**: Proporcionamos mousepads de diferentes materiales y tamaños para mejorar la comodidad durante el uso del mouse.\n\nSi tienes alguna pregunta específica o necesitas más información sobre estos productos, no dudes en decírmelo.', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-10-19T20:26:13.046717

In [5]:
from langchain_core.tools import tool
import requests

@tool("get_products", description="Get the products that the store offers filter by price")
def get_products():
    # Connnect with API
    """Get the products that the store offers"""
    response = requests.get("https://api.escuelajs.co/api/v1/products")
    products = response.json()
    return "".join([f"{product['title']} - {product['price']}" for product in products])



In [6]:
get_products.invoke({})

'Artss - 11122Artix - 123123Alpha1760905210304 - 100Furniture product_RHWV - 10Furniture product - 10Furniture product_9L7X - 10Furniture product_JVHW - 10'

In [7]:
@tool("get_weather", description="Get the weather of a city")
def get_weather(city: str):
    response = requests.get(f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1")
    data = response.json()
    latitude = data["results"][0]["latitude"]
    longitude = data["results"][0]["longitude"]
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true")
    data = response.json()
    response = f"The weather in {city} is {data["current_weather"]["temperature"]}C with {data["current_weather"]["windspeed"]}km/h of wind."
    return response

get_weather.invoke({"city": "bogota"})

'The weather in bogota is 16.2C with 12.7km/h of wind.'

In [8]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que necesitan y dar el clima de la ciudad

Tus tools son:
- get_products: para obtener los productos que ofreces en la tienda
- get_weather: para obtener el clima de la ciudad
"""
messages = [
    ("system", system_prompt),
    ("user", "Dime los productos que ofreces en la tienda")
]
llm_with_tools = llm.bind_tools([get_products, get_weather])
response = llm_with_tools.invoke(messages)
response.tool_calls

[]

In [9]:
messages = [
    ("system", system_prompt),
    ("user", "Hola, que tal?")
]
response = llm_with_tools.invoke(messages)
response.text

<bound method BaseMessage.text of AIMessage(content='¡Hola! Estoy bien, gracias. ¿En qué puedo ayudarte hoy?', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-10-19T20:26:19.155616993Z', 'done': True, 'done_reason': 'stop', 'total_duration': 796752926, 'load_duration': 50496654, 'prompt_eval_count': 259, 'prompt_eval_duration': 19555529, 'eval_count': 18, 'eval_duration': 722853690, 'model_name': 'qwen2.5:7b'}, id='run--15238c23-7718-4e7a-b87c-8c842a481e4e-0', usage_metadata={'input_tokens': 259, 'output_tokens': 18, 'total_tokens': 277})>

In [10]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que necesitan y dar el clima de la ciudad

Tus tools son:
- get_products: para obtener los productos que ofreces en la tienda
- get_weather: para obtener el clima de la ciudad
"""
messages = [
    ("system", system_prompt),
    ("user", "Cual es el clima en la capital de Bolivia?")
]
llm_with_tools = llm.bind_tools([get_products, get_weather])
response = llm_with_tools.invoke(messages)
response.tool_calls

[]